In [1]:
# NHSRC PHC SUPPLY CHAIN - DATA CLEANING PIPELINE
import pandas as pd
import numpy as np

print("🧹 NHSRC PHC DATA CLEANING PIPELINE")
print("=" * 50)

🧹 NHSRC PHC DATA CLEANING PIPELINE


In [3]:
# 1. IMPORT AND BASIC READ
print("📊 STEP 1: Importing Data")
print("-" * 40)

inventory = pd.read_csv("data/sample_inventory.csv")
master = pd.read_csv("data/medicine_master.csv")

print("📦 INVENTORY DATA:")
print(f"Records: {len(inventory):,}")
print(f"Columns: {list(inventory.columns)}")
display(inventory.head())

print("\n🏥 MASTER DATA:")
print(f"Records: {len(master):,}")
print(f"Columns: {list(master.columns)}")
display(master.head())

📊 STEP 1: Importing Data
----------------------------------------
📦 INVENTORY DATA:
Records: 6,480
Columns: ['date', 'facility_id', 'facility_name', 'ward_id', 'sku_id', 'sku_name', 'units_used', 'on_hand', 'stock_in_transit', 'lead_time_days', 'batch_id', 'batch_expiry_date', 'price_per_unit', 'is_holiday', 'patient_admissions', 'VED_category', 'FSN_category']


,date,facility_id,facility_name,ward_id,sku_id,sku_name,units_used,on_hand,stock_in_transit,lead_time_days,batch_id,batch_expiry_date,price_per_unit,is_holiday,patient_admissions,VED_category,FSN_category
0,2024-01-01,FAC001,Urban PHC Kalyan,W03,MED001,Paracetamol Tablet 500mg,36,398,0,3,B0011753,2024-06-29,31.53,0,128,Vital,Fast
1,2024-01-01,FAC001,Urban PHC Kalyan,W03,MED002,Amoxicillin Capsule 250mg,27,232,0,2,B0021207,2024-03-31,26.05,0,82,Vital,Fast
2,2024-01-01,FAC001,Urban PHC Kalyan,W03,MED003,Oral Rehydration Salts,15,257,0,4,B0039863,2024-12-31,37.74,0,101,Vital,Fast
3,2024-01-01,FAC001,Urban PHC Kalyan,W03,MED004,Ibuprofen Tablet 200mg,20,492,0,2,B0045952,2025-12-31,19.58,0,128,Essential,Fast
4,2024-01-01,FAC001,Urban PHC Kalyan,W01,MED005,Cetirizine Tablet 5mg,54,351,0,2,B0051421,2024-03-31,19.65,0,93,Essential,Fast



🏥 MASTER DATA:
Records: 12
Columns: ['sku_id', 'sku_name', 'VED_category', 'FSN_category', 'unit', 'base_consumption', 'seasonality', 'weekend_effect']


,sku_id,sku_name,VED_category,FSN_category,unit,base_consumption,seasonality,weekend_effect
0,MED001,Paracetamol Tablet 500mg,Vital,Fast,Tablet,25,1.3,0.80
1,MED002,Amoxicillin Capsule 250mg,Vital,Fast,Capsule,20,1.4,0.70
2,MED003,Oral Rehydration Salts,Vital,Fast,Packet,15,1.5,0.90
3,MED004,Ibuprofen Tablet 200mg,Essential,Fast,Tablet,18,1.2,0.85
4,MED005,Cetirizine Tablet 5mg,Essential,Fast,Tablet,12,1.1,0.90


In [5]:
# 2. STANDARDIZE COLUMN NAMES
print("\n📝 STEP 2: Standardizing Column Names")
print("-" * 40)

print("BEFORE standardization:")
print(f"Inventory columns: {list(inventory.columns)}")
print(f"Master columns: {list(master.columns)}")

# Standardize column names
inventory.columns = (
    inventory.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)

master.columns = (
    master.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)

print("\nAFTER standardization:")
print(f"Inventory columns: {list(inventory.columns)}")
print(f"Master columns: {list(master.columns)}")


📝 STEP 2: Standardizing Column Names
----------------------------------------
BEFORE standardization:
Inventory columns: ['date', 'facility_id', 'facility_name', 'ward_id', 'sku_id', 'sku_name', 'units_used', 'on_hand', 'stock_in_transit', 'lead_time_days', 'batch_id', 'batch_expiry_date', 'price_per_unit', 'is_holiday', 'patient_admissions', 'VED_category', 'FSN_category']
Master columns: ['sku_id', 'sku_name', 'VED_category', 'FSN_category', 'unit', 'base_consumption', 'seasonality', 'weekend_effect']

AFTER standardization:
Inventory columns: ['date', 'facility_id', 'facility_name', 'ward_id', 'sku_id', 'sku_name', 'units_used', 'on_hand', 'stock_in_transit', 'lead_time_days', 'batch_id', 'batch_expiry_date', 'price_per_unit', 'is_holiday', 'patient_admissions', 'ved_category', 'fsn_category']
Master columns: ['sku_id', 'sku_name', 'ved_category', 'fsn_category', 'unit', 'base_consumption', 'seasonality', 'weekend_effect']


In [7]:
# 3. FIX DATE FORMATS - SIMPLE & CORRECT
print("\n📅 STEP 3: Fixing Date Formats")
print("-" * 40)

print("Date columns before conversion:")
print(f"date dtype: {inventory['date'].dtype}")
print(f"batch_expiry_date dtype: {inventory['batch_expiry_date'].dtype}")

# Convert dates - they're in YYYY-MM-DD format
inventory['date'] = pd.to_datetime(inventory['date'], format='%Y-%m-%d', errors='raise')
inventory['batch_expiry_date'] = pd.to_datetime(inventory['batch_expiry_date'], format='%Y-%m-%d', errors='raise')

print("\n✅ Date conversion successful!")
print(f"date dtype: {inventory['date'].dtype}")
print(f"batch_expiry_date dtype: {inventory['batch_expiry_date'].dtype}")
print(f"Date range: {inventory['date'].min()} to {inventory['date'].max()}")
print(f"Total days: {(inventory['date'].max() - inventory['date'].min()).days + 1} days")

# Quick sanity check
print(f"\n🧪 Sanity checks:")
print(f"  - Any null dates? {inventory['date'].isna().sum()}")
print(f"  - Any null expiry dates? {inventory['batch_expiry_date'].isna().sum()}")
print(f"  - Any expiry dates before today? {(inventory['batch_expiry_date'] < inventory['date']).sum()}")


📅 STEP 3: Fixing Date Formats
----------------------------------------
Date columns before conversion:
date dtype: object
batch_expiry_date dtype: object

✅ Date conversion successful!
date dtype: datetime64[ns]
batch_expiry_date dtype: datetime64[ns]
Date range: 2024-01-01 00:00:00 to 2024-06-28 00:00:00
Total days: 180 days

🧪 Sanity checks:
  - Any null dates? 0
  - Any null expiry dates? 0
  - Any expiry dates before today? 0


In [11]:
# Find dates that aren't standard YYYY-MM-DD
print("🔍 LOOKING FOR NON-STANDARD DATE FORMATS")
print("=" * 50)

df_raw = pd.read_csv('data/sample_inventory.csv')
unique_dates = df_raw['date'].unique()

print(f"Total unique dates: {len(unique_dates)}")
print("\nFirst 20 unique dates:")
for i, date_str in enumerate(unique_dates[:20]):
    print(f"  {i:3}: '{date_str}'")

# Check for dates with length != 10
print("\n📏 CHECKING DATE STRING LENGTHS:")
length_counts = df_raw['date'].astype(str).str.len().value_counts().sort_index()
for length, count in length_counts.items():
    print(f"  Length {length}: {count} rows")

# If any length != 10, show them
non_standard = df_raw[df_raw['date'].astype(str).str.len() != 10]
if len(non_standard) > 0:
    print(f"\n🚨 FOUND {len(non_standard)} ROWS WITH NON-STANDARD DATE LENGTH:")
    display(non_standard[['date', 'sku_id', 'facility_id']].head(10))

🔍 LOOKING FOR NON-STANDARD DATE FORMATS
Total unique dates: 180

First 20 unique dates:
    0: '2024-01-01'
    1: '2024-01-02'
    2: '2024-01-03'
    3: '2024-01-04'
    4: '2024-01-05'
    5: '2024-01-06'
    6: '2024-01-07'
    7: '2024-01-08'
    8: '2024-01-09'
    9: '2024-01-10'
   10: '2024-01-11'
   11: '2024-01-12'
   12: '2024-01-13'
   13: '2024-01-14'
   14: '2024-01-15'
   15: '2024-01-16'
   16: '2024-01-17'
   17: '2024-01-18'
   18: '2024-01-19'
   19: '2024-01-20'

📏 CHECKING DATE STRING LENGTHS:
  Length 10: 6480 rows


In [9]:
# 4. HANDLE MISSING VALUES
print("\n🔍 STEP 4: Handling Missing Values")
print("-" * 40)

print("Missing values BEFORE handling:")
missing_before = inventory.isna().sum()
print(missing_before[missing_before > 0])

# Handle missing values based on column type
# IDs: Drop if missing (none in our case)
# Units Used: Replace with 0
inventory['units_used'] = inventory['units_used'].fillna(0)

# Stock: Forward fill by SKU
inventory['on_hand'] = inventory.groupby('sku_id')['on_hand'].fillna(method='ffill')

# Lead Time: Fill with median by SKU
inventory['lead_time_days'] = inventory.groupby('sku_id')['lead_time_days'].transform(
    lambda x: x.fillna(x.median())
)

print("\nMissing values AFTER handling:")
missing_after = inventory.isna().sum()
print(missing_after[missing_after > 0])


🔍 STEP 4: Handling Missing Values
----------------------------------------
Missing values BEFORE handling:
Series([], dtype: int64)

Missing values AFTER handling:
Series([], dtype: int64)


C:\Users\wicra\AppData\Local\Temp\ipykernel_9100\1934835610.py:15: FutureWarning: SeriesGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use Series.fillna instead
  inventory['on_hand'] = inventory.groupby('sku_id')['on_hand'].fillna(method='ffill')
C:\Users\wicra\AppData\Local\Temp\ipykernel_9100\1934835610.py:15: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  inventory['on_hand'] = inventory.groupby('sku_id')['on_hand'].fillna(method='ffill')


In [13]:
print(master.columns.tolist())

['sku_id', 'sku_name', 'ved_category', 'fsn_category', 'unit', 'base_consumption', 'seasonality', 'weekend_effect']


In [15]:
print(inventory.columns.tolist())

['date', 'facility_id', 'facility_name', 'ward_id', 'sku_id', 'sku_name', 'units_used', 'on_hand', 'stock_in_transit', 'lead_time_days', 'batch_id', 'batch_expiry_date', 'price_per_unit', 'is_holiday', 'patient_admissions', 'ved_category', 'fsn_category']


In [17]:
# both master df and inventory df has ved_category and fsn_category so we have to drop these columns from inventory 
# to reduce further conflicts with the workflow
inventory.drop(["ved_category", "fsn_category"], axis = 1, inplace=True)

In [19]:
# 5. STANDARDIZE SKU → MASTER MERGE CHECK
print("\n🔗 STEP 5: SKU-Master Merge Validation")
print("-" * 40)

# Merge with master data
merged = inventory.merge(master[['sku_id', 'ved_category', 'fsn_category']], on="sku_id", how="left")

# Check for missing master records
missing_master = merged[merged['ved_category'].isna()]
print(f"Records missing master data: {len(missing_master)}")

if len(missing_master) > 0:
    print("Missing master records:")
    display(missing_master[['sku_id', 'sku_name']].drop_duplicates())
else:
    print("✅ All SKUs have corresponding master data")

# Use the merged data
inventory = merged


🔗 STEP 5: SKU-Master Merge Validation
----------------------------------------
Records missing master data: 0
✅ All SKUs have corresponding master data


In [21]:
# 6. OUTLIER DETECTION
print("\n🚨 STEP 6: Outlier Detection")
print("-" * 40)

# Flag outliers: demand > mean + 3*std
inventory['is_outlier'] = (
    inventory.groupby('sku_id')['units_used']
    .transform(lambda x: (x > x.mean() + 3 * x.std()))
)

outlier_summary = inventory.groupby('sku_id')['is_outlier'].sum()
print("Outlier count by SKU:")
for sku, count in outlier_summary[outlier_summary > 0].items():
    print(f"  {sku}: {count} outliers")

total_outliers = inventory['is_outlier'].sum()
print(f"\nTotal outliers flagged: {total_outliers} (not removed - important for outbreak signals)")


🚨 STEP 6: Outlier Detection
----------------------------------------
Outlier count by SKU:
  MED001: 10 outliers
  MED002: 10 outliers
  MED003: 9 outliers
  MED004: 12 outliers
  MED005: 10 outliers
  MED006: 15 outliers
  MED007: 17 outliers
  MED008: 10 outliers
  MED009: 14 outliers
  MED010: 14 outliers
  MED011: 15 outliers
  MED012: 9 outliers

Total outliers flagged: 145 (not removed - important for outbreak signals)


In [23]:
# 7. COMPUTE DERIVED FIELDS
print("\n🧮 STEP 7: Computing Derived Fields")
print("-" * 40)

# Days Cover: Stock adequacy (7-day rolling average)
inventory['days_cover'] = inventory['on_hand'] / (inventory['units_used'].rolling(7).mean().replace(0, np.nan))
inventory['days_cover'] = inventory['days_cover'].fillna(365)  # Fill infinite values

# Expiry days remaining
inventory['expiry_days_remaining'] = (inventory['batch_expiry_date'] - inventory['date']).dt.days

# Expiry risk bucket (NHSRC compliance)
inventory['expiry_risk_bucket'] = pd.cut(
    inventory['expiry_days_remaining'],
    bins=[-999, 30, 90, 180, 9999],
    labels=["CRITICAL (<30d)", "HIGH (30–90d)", "MEDIUM (90–180d)", "LOW (>180d)"]
)

print("✅ Derived fields computed:")
print(f"  - Days Cover: Stock adequacy metric")
print(f"  - Expiry Days Remaining: {inventory['expiry_days_remaining'].min()} to {inventory['expiry_days_remaining'].max()} days")
print(f"  - Expiry Risk Bucket: NHSRC compliance categories")

# Show expiry risk distribution
expiry_dist = inventory['expiry_risk_bucket'].value_counts()
print("\n📊 Expiry Risk Bucket Distribution:")
for risk, count in expiry_dist.items():
    print(f"  {risk}: {count} records")


🧮 STEP 7: Computing Derived Fields
----------------------------------------
✅ Derived fields computed:
  - Days Cover: Stock adequacy metric
  - Expiry Days Remaining: 30 to 730 days
  - Expiry Risk Bucket: NHSRC compliance categories

📊 Expiry Risk Bucket Distribution:
  LOW (>180d): 2636 records
  MEDIUM (90–180d): 1874 records
  HIGH (30–90d): 1633 records
  CRITICAL (<30d): 337 records


In [25]:
# 8. DATA QUALITY SUMMARY
print("\n📋 STEP 8: Data Quality Summary")
print("-" * 40)

report = {
    "total_records": len(inventory),
    "date_range": f"{inventory['date'].min().date()} to {inventory['date'].max().date()}",
    "unique_facilities": inventory['facility_id'].nunique(),
    "unique_skus": inventory['sku_id'].nunique(),
    "missing_values_total": inventory.isna().sum().sum(),
    "outlier_count": int(inventory['is_outlier'].sum()),
    "ved_distribution": inventory['ved_category'].value_counts().to_dict(),
    "fsn_distribution": inventory['fsn_category'].value_counts().to_dict(),
    "expiry_risk_distribution": inventory['expiry_risk_bucket'].value_counts().to_dict()
}

print("📈 DATA QUALITY REPORT:")
for key, value in report.items():
    print(f"  {key}: {value}")


📋 STEP 8: Data Quality Summary
----------------------------------------
📈 DATA QUALITY REPORT:
  total_records: 6480
  date_range: 2024-01-01 to 2024-06-28
  unique_facilities: 3
  unique_skus: 12
  missing_values_total: 0
  outlier_count: 145
  ved_distribution: {'Vital': 2700, 'Essential': 2160, 'Desirable': 1620}
  fsn_distribution: {'Fast': 3240, 'Slow': 3240}
  expiry_risk_distribution: {'LOW (>180d)': 2636, 'MEDIUM (90–180d)': 1874, 'HIGH (30–90d)': 1633, 'CRITICAL (<30d)': 337}


In [27]:
# 9. SAVE CLEANED DATASET
print("\n💾 STEP 9: Saving Cleaned Dataset")
print("-" * 40)

inventory.to_csv("data/cleaned_inventory.csv", index=False)

print("✅ Cleaned data saved to: data/cleaned_inventory.csv")
print(f"📁 File info:")
print(f"  - Records: {len(inventory):,}")
print(f"  - Columns: {len(inventory.columns)}")
print(f"  - Size: {inventory.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")

# Display first 5 rows of cleaned data
print("\n🔹 FIRST 5 ROWS OF CLEANED_INVENTORY.CSV:")
display(inventory.head())


💾 STEP 9: Saving Cleaned Dataset
----------------------------------------
✅ Cleaned data saved to: data/cleaned_inventory.csv
📁 File info:
  - Records: 6,480
  - Columns: 21
  - Size: 3.45 MB

🔹 FIRST 5 ROWS OF CLEANED_INVENTORY.CSV:


,date,facility_id,facility_name,ward_id,sku_id,sku_name,units_used,on_hand,stock_in_transit,lead_time_days,...,batch_expiry_date,price_per_unit,is_holiday,patient_admissions,ved_category,fsn_category,is_outlier,days_cover,expiry_days_remaining,expiry_risk_bucket
0,2024-01-01,FAC001,Urban PHC Kalyan,W03,MED001,Paracetamol Tablet 500mg,36,398,0,3,...,2024-06-29,31.53,0,128,Vital,Fast,False,365.0,180,MEDIUM (90–180d)
1,2024-01-01,FAC001,Urban PHC Kalyan,W03,MED002,Amoxicillin Capsule 250mg,27,232,0,2,...,2024-03-31,26.05,0,82,Vital,Fast,False,365.0,90,HIGH (30–90d)
2,2024-01-01,FAC001,Urban PHC Kalyan,W03,MED003,Oral Rehydration Salts,15,257,0,4,...,2024-12-31,37.74,0,101,Vital,Fast,False,365.0,365,LOW (>180d)
3,2024-01-01,FAC001,Urban PHC Kalyan,W03,MED004,Ibuprofen Tablet 200mg,20,492,0,2,...,2025-12-31,19.58,0,128,Essential,Fast,False,365.0,730,LOW (>180d)
4,2024-01-01,FAC001,Urban PHC Kalyan,W01,MED005,Cetirizine Tablet 5mg,54,351,0,2,...,2024-03-31,19.65,0,93,Essential,Fast,True,365.0,90,HIGH (30–90d)


In [29]:
# 10. GENERATE DATA QUALITY REPORT
print("\n📄 STEP 10: Generating Data Quality Report")
print("-" * 40)

report_content = f"""# NHSRC PHC Supply Chain - Data Quality Report

## Dataset Overview
- **Total Records**: {report['total_records']:,}
- **Date Range**: {report['date_range']}
- **Unique Facilities**: {report['unique_facilities']}
- **Unique SKUs**: {report['unique_skus']}

## Data Quality Metrics
- **Missing Values**: {report['missing_values_total']}
- **Outliers Flagged**: {report['outlier_count']}

## Category Distributions

### VED Category Distribution
{pd.Series(report['ved_distribution'])}

### FSN Category Distribution  
{pd.Series(report['fsn_distribution'])}

### Expiry Risk Distribution
{pd.Series(report['expiry_risk_distribution'])}

## Cleaning Operations Applied
1. Column name standardization
2. Date format conversion
3. Missing value imputation
4. Outlier detection (flagged, not removed)
5. Derived field computation
6. Master data validation

## Notes
- Outliers are flagged but not removed to preserve outbreak signals
- All SKUs have corresponding master data
- Data is NHSRC-compliant and ready for analysis

**Generated**: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}
"""

# Save report
with open('docs/data_quality_report.md', 'w') as f:
    f.write(report_content)

print("✅ Data quality report saved to: docs/data_quality_report.md")


📄 STEP 10: Generating Data Quality Report
----------------------------------------
✅ Data quality report saved to: docs/data_quality_report.md


In [31]:
df = pd.read_csv('data/cleaned_inventory.csv')
print(f"Missing dates: {df['date'].isna().sum()}")

Missing dates: 0


In [35]:
df.head(10)

,date,facility_id,facility_name,ward_id,sku_id,sku_name,units_used,on_hand,stock_in_transit,lead_time_days,...,batch_expiry_date,price_per_unit,is_holiday,patient_admissions,ved_category,fsn_category,is_outlier,days_cover,expiry_days_remaining,expiry_risk_bucket
0,2024-01-01,FAC001,Urban PHC Kalyan,W03,MED001,Paracetamol Tablet 500mg,36,398,0,3,...,2024-06-29,31.53,0,128,Vital,Fast,False,365.000000,180,MEDIUM (90–180d)
1,2024-01-01,FAC001,Urban PHC Kalyan,W03,MED002,Amoxicillin Capsule 250mg,27,232,0,2,...,2024-03-31,26.05,0,82,Vital,Fast,False,365.000000,90,HIGH (30–90d)
2,2024-01-01,FAC001,Urban PHC Kalyan,W03,MED003,Oral Rehydration Salts,15,257,0,4,...,2024-12-31,37.74,0,101,Vital,Fast,False,365.000000,365,LOW (>180d)
3,2024-01-01,FAC001,Urban PHC Kalyan,W03,MED004,Ibuprofen Tablet 200mg,20,492,0,2,...,2025-12-31,19.58,0,128,Essential,Fast,False,365.000000,730,LOW (>180d)
4,2024-01-01,FAC001,Urban PHC Kalyan,W01,MED005,Cetirizine Tablet 5mg,54,351,0,2,...,2024-03-31,19.65,0,93,Essential,Fast,True,365.000000,90,HIGH (30–90d)
5,2024-01-01,FAC001,Urban PHC Kalyan,W01,MED006,Insulin Injection 40 IU/ml,2,29,0,2,...,2024-06-29,42.42,0,120,Vital,Slow,False,365.000000,180,MEDIUM (90–180d)
6,2024-01-01,FAC001,Urban PHC Kalyan,W03,MED007,Adrenaline Injection 1 mg/ml,1,47,0,3,...,2025-12-31,33.05,0,123,Vital,Slow,False,2.122581,730,LOW (>180d)
7,2024-01-01,FAC001,Urban PHC Kalyan,W02,MED008,Omeprazole Capsule 20mg,6,61,0,3,...,2024-06-29,3.13,0,108,Essential,Slow,False,3.416000,180,MEDIUM (90–180d)
8,2024-01-01,FAC001,Urban PHC Kalyan,W01,MED009,Metformin Tablet 500mg,11,74,39,5,...,2024-06-29,12.48,0,129,Essential,Slow,False,4.752294,180,MEDIUM (90–180d)
9,2024-01-01,FAC001,Urban PHC Kalyan,W02,MED010,Multivitamin Tablet,6,75,0,3,...,2024-12-31,6.09,0,71,Desirable,Slow,False,5.250000,365,LOW (>180d)
